Load common word vectors

In [ ]:
import csv
with open('common_word_vectors.csv', 'r') as file:
  reader = csv.DictReader(file)
  common_words = {k: list(map(int, v.split(','))) for k, v in next(reader).items()}

Load image to text

In [ ]:
from transformers import pipeline

In [ ]:
captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base")

Load cosine distance

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
def cos(word1, word2):
  return cosine_similarity([vectors[word1]], [vectors[word2]])[0][0]

In [ ]:
def closest(n, word, dct):
  wordvec = dct[word]
  lst=[]
  for key, value in dct.items():
    lst.append([key,cosine_similarity([wordvec],[value])[0][0]])
    lst.sort(key=lambda x: x[1], reverse=True)
    if len(lst) > n:
      lst = lst[:n]
  return lst

Create sentence splitter and joiner

In [ ]:
import re

In [ ]:
def sentence_to_list(sentence):
  # Remove punctuation from the sentence
  sentence_no_punctuation = re.sub(r'[^\w\s]', '', sentence)
  # Split the sentence into words
  words = sentence_no_punctuation.split()
  return words

In [ ]:
def list_to_sentence(l):
  return ' '.join(l)

Load image generator

In [ ]:
!pip install bingart -q

In [ ]:
from bingart import BingArt
from PIL import Image
import requests
from io import BytesIO

# Getting authentication
*   Go to https://bing.com/.
*   F12 to open console
*   In the JavaScript console, type cookieStore.get("_U").then(result => console.log(result.value)) and press enter
*   Copy the output. This is used in --U or auth_cookie.
### Firefox
*   Go to https://bing.com/.
*   F12 to open developer tools
*   navigate to the storage tab
*   expand the cookies tab
*   click on the https://bing.com cookie
*   copy the value from the _U

In [ ]:
auth_cookie_U='paste your cookie here'

In [ ]:
bing_art = BingArt(auth_cookie_U=auth_cookie_U)

In [ ]:
def get_bing_image(prompt):
  try:
    results = bing_art.generate_images(prompt)
  finally:
    bing_art.close_session()
  url = results['images'][0]['url']
  # Send a GET request to the image URL
  response = requests.get(url)
  # Open the image from the response content
  img = Image.open(BytesIO(response.content))
  return img

In [ ]:
ignore_short_words=['and','the','or','a','an','with','in','of','on','between','next','to','around','it','its','top']

In [ ]:
import random

In [ ]:
def mutate(sentence, d, r): # d is how far to mutate minimum. r is how much randomness to add to d each time.
  l = sentence_to_list(sentence)
  for i in range(len(l)):
    if len(l[i]) >= 3 and l[i] not in ignore_short_words:
      d2 = d + random.randint(0,r)
      if l[i] in common_words:
        l[i] = closest(d2+1, l[i], common_words)[d2][0]
      elif l[i][-1] == 's' and l[i][:-1] in common_words: # sometimes plurals are not in but the singular is. Handle it like this.
        l[i] = closest(d2+1, l[i][:-1], common_words)[d2][0]+'s'
      else:
        l[i] = ''
  output = list_to_sentence(l)
  output = output.replace('_', ' ') # Replace underscores with spaces
  return output

In [ ]:
def paper_game(prompt, n, d, r): # n is how many images. d is how far to mutate minimum. r is how much randomness to add to d each time.
  print("Prompt: ",prompt)
  try:
    for i in range(n):
      image = get_bing_image(prompt)
      display(image)
      if i >= n-1:
        break
      text=captioner(image, max_new_tokens=20)[0]['generated_text']
      print("AI generated caption: ",text)
      prompt = mutate(text, d, r)
      print("Mutated caption: ",prompt)
  except Exception:
    print('Prompt rejected by Bing')

In [ ]:
paper_game('That elephant is causing the fox some concern', 8, 15, 5)